# Integrating tools/function with LLM using Azure OpenAI

In [1]:
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [14]:
# Define constants for the Azure OpenAI API configuration
api_key = "xxxxxxxxxxxxxxxxx"
api_version = "2023-07-01-preview" # "2023-05-15"
azure_endpoint = "https://xxxxxxxxxx.openai.azure.com/"
model_name = "gpt-35-turbo"

In [15]:
import os
from openai import AzureOpenAI
import json
import requests

client = AzureOpenAI(
  azure_endpoint = azure_endpoint, 
  api_key=api_key,  
  api_version=api_version
)


In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

## Supported models
Not all model versions are trained with function calling data. Function calling is supported with the following models: gpt-4, gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4-0613, gpt-3.5-turbo, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, and gpt-3.5-turbo-0613

In addition, parallel function calls is supported on the following models: gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-3.5-turbo-0125, and gpt-3.5-turbo-1106

In [16]:
# Define the OpenWeatherMap API key
OWM_API_KEY = "29af1cea50a401d8e624eea4660b3f59"

def get_current_weather(location, unit="kelvin"):
    """
    Fetches the current weather information for a given location.

    Parameters:
    - location: str, the name of the location (e.g., "Paris").
    - unit: str, the unit of temperature (default is "kelvin").

    Returns:
    - str: JSON formatted string containing weather information.
    """
    # Construct the API request URL
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={OWM_API_KEY}"
    
    # Send the API request
    response = requests.get(url)
    
    # Parse the temperature and weather forecast from the response
    temp = response.json()['main']['temp']
    forecast = [response.json()['weather'][0]['main'], response.json()['weather'][0]['description']]

    # Create a dictionary with the weather information
    weather_info = {
        "location": location,
        "temperature": temp,
        "unit": 'Kelvin',
        "forecast": forecast
    }
    
    # Return the weather information as a JSON string
    return json.dumps(weather_info)

In [17]:
get_current_weather("Paris")

'{"location": "Paris", "temperature": 293.84, "unit": "Kelvin", "forecast": ["Clear", "clear sky"]}'

In [18]:
get_current_weather("Manila")

'{"location": "Manila", "temperature": 302.84, "unit": "Kelvin", "forecast": ["Clouds", "scattered clouds"]}'

In [20]:
messages = [{"role":"user",'content':"what is AI?"}]
results = client.chat.completions.create(model = model_name, messages = messages)
print(results.model_dump_json(indent=2))

{
  "id": "chatcmpl-9ZgZgS218VdSSMvUsCS9AxAUEAMXq",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "AI (Artificial Intelligence) is a branch of computer science that focuses on creating smart machines capable of performing tasks that typically require human intelligence. AI systems are designed to simulate human intelligence, including the ability to learn, reason, solve problems, understand language, and perceive and respond to their environments. AI technologies can be categorized into narrow AI, which is designed for specific tasks like facial recognition or language translation, and general AI, which aims to possess the same intellectual capabilities as a human.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": 

In [22]:

# Define tools/functions for the LLM to use
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

In [26]:
def get_response(messages, tools, model=model_name):
    """
    Handles interaction with the LLM, including making function calls if needed.

    Parameters:
    - messages: list, the conversation history.
    - tools: list, the functions/tools available to the LLM.
    - model: str, the model name to use (default is model_name).

    Returns:
    - str: The response from the LLM or function.
    """
    # Create a chat completion with the given messages and tools
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice='auto',
        temperature=0.5,
    )

    # Get the generated response and tool calls (if any)
    response = response.choices[0].message
    tool_calls = response.tool_calls

    try:
        if tool_calls:
            print("Making a function call")
            # Available functions
            available_functions = {
                "get_current_weather": get_current_weather,
            }
            print(tool_calls)

            # Extend conversation with assistant's reply
            messages.append(response)

            # Handle each function call
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(
                    location=function_args.get("location"),
                    unit=function_args.get("unit"),
                )
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

            # Get a new response from the model with the function response
            second_response = client.chat.completions.create(
                model=model_name,
                messages=messages,
            )
            return second_response
        else:
            return response.content
    except Exception as e:
        print("Error occurred", e)
        return response

In [27]:
# Example usage of get_response function
messages = [{"role": "user", "content": "Provide a 2 line explanation for AI"}]
response = get_response(messages, tools)
print(response)

AI, or Artificial Intelligence, is a branch of computer science that focuses on creating intelligent machines that can perform tasks that would typically require human intelligence. It involves the development of algorithms and models that enable machines to learn, reason, and make decisions.


In [25]:
messages = [{"role": "user", "content": "How is the weather in Tokyo today?"}]
response = get_response(messages, tools)
print(response.choices[0].message.content)

Making a function call
[ChatCompletionMessageToolCall(id='call_7laQPyXq7XwZmqbit3kwYaS9', function=Function(arguments='{\n  "location": "Tokyo"\n}', name='get_current_weather'), type='function')]
The weather in Tokyo today is clear with a temperature of 295.26 Kelvin.
